In [15]:
import pandas as pd
import seaborn as sns
from collections import Counter
import matplotlib as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [16]:
po_path = "/shares/wegner.ics.uzh/glc23_data/Presences_only_train.csv"
pa_path = "/shares/wegner.ics.uzh/glc23_data/Presence_Absence_surveys/Presences_Absences_train.csv"
p1 = "/shares/wegner.ics.uzh/glc23_data/Pot_10_to_1000.csv"
p2 = "/shares/wegner.ics.uzh/glc23_data/po_none_to_1000.csv"
p3 = "/shares/wegner.ics.uzh/glc23_data/po_10_to_1000.csv"
p4 = "/shares/wegner.ics.uzh/glc23_data/po_10_to_none.csv"
pa24_path = "/shares/wegner.ics.uzh/glc24_data/PresenceAbsenceSurveys/GLC24-PA-metadata-train.csv"
po24_path = "/shares/wegner.ics.uzh/glc24_data/PresenceOnlyOccurrences/GLC24-PO-metadata-train.csv"

In [5]:
d1 = pd.read_csv(p1, sep=";")
d2 = pd.read_csv(p2, sep=";")
d3 = pd.read_csv(p3, sep=";")
d4 = pd.read_csv(p4, sep=";")

FileNotFoundError: [Errno 2] No such file or directory: '/shares/wegner.ics.uzh/glc23_data/po_none_1000.csv'

In [6]:
po_ds = pd.read_csv(po_path, sep=";")
pa_ds = pd.read_csv(pa_dataset_path, sep=";")

In [ ]:
pa_ds24 = pd.read_csv(pa24_path, sep=",")
po_ds24 = pd.read_csv(po24_path, sep=",")

### Deciding on the val/test partition

In [17]:
len(po_ds)

4908319

In [18]:
len(pa_ds)

85325

In [50]:
val, test = train_test_split(pa_ds, test_size=0.4, random_state=20)
val_ids = val["speciesId"].unique()
test_ids = test["speciesId"].unique()

In [51]:
bins = [0, 10, 20, 60, 100, 500, 1000, 5000]
species_counts = po_ds["speciesId"].value_counts()
grouped_species_ids = pd.cut(species_counts, bins=bins, labels=[f'{start}-{end}' for start, end in zip(bins[:-1], bins[1:])])

In [52]:
ids_dic = {}
for bin in grouped_species_ids.unique():
    ids_dic[bin] = grouped_species_ids[grouped_species_ids==bin].index\
        .to_list()

In [53]:
summary = {}
train_counts = {}
val_counts = {}
test_counts = {}

for range, ids in ids_dic.items():
    # ensure that we only calculate the f1 for the species that are in the test set
    int_val_ids= list(set(ids).intersection(set(val_ids)))
    int_test_ids = list(set(ids).intersection(set(test_ids)))
    train_counts[range] = len(ids)
    val_counts[range] = len(int_val_ids)
    test_counts[range] = len(int_test_ids)

In [55]:
pd.DataFrame([train_counts, val_counts, test_counts], index=["train", "val", "test"])

,1000-5000,500-1000,100-500,60-100,20-60,10-20,0-10
train,1319,440,1313,435,1202,860,4469
val,931,191,382,79,161,74,196
test,873,173,325,75,129,62,160


### Intersection between datasets

In [6]:
len(set(po_ds["speciesId"]) - set(pa_ds["speciesId"]))

7864

In [13]:
len(set(po_ds["speciesId"]) - set(pa_ds["speciesId"]))

7864

In [14]:
len(set(po_ds["speciesId"]).intersection(set(pa_ds["speciesId"])))

2174

In [15]:
len(set(po_ds["speciesId"]).intersection(set(pa_ds24["speciesId"])))

4454

In [20]:
len(set(po_ds["speciesId"]).intersection(set(po_ds24["speciesId"])))

8666

In [26]:
len(set(pa_ds["speciesId"]))

2174

In [21]:
len(set(pa_ds24["speciesId"]))

5016

In [23]:
len(set(po_ds["speciesId"]))

10038

In [24]:
po_ds["speciesId"].max()

10039

In [22]:
len(set(po_ds24["speciesId"]))

9709

In [32]:
po_ds24["speciesId"].min()

0.0

In [33]:
pa_ds24["speciesId"].min()

2.0

In [34]:
pa_ds24["speciesId"].max()

11254.0

In [16]:
len(pa_ds24["speciesId"].unique())

5016

In [35]:
len(set(pa_ds24["speciesId"]).intersection(set(po_ds24["speciesId"])))

4367

In [25]:
# Code-Snippet to match GLC23 occurrence with gbif occurrence to retrieve the original species
print(po_ds[po_ds["speciesId"] == 265].iloc[0])

glcID                           4813964
gbifID                     3956390831.0
observer                            NaN
datasetName          Pl@ntNet automatic
date                         2018-09-04
dayOfYear                           247
year                               2018
lon                           -9.228707
lat                            38.71163
x_EPSG3035                    2657705.0
y_EPSG3035                    1947568.0
geoUncertaintyInM                 13.78
speciesId                           265
patchID                         3026669
timeSerieID                        8808
Name: 7975, dtype: object


In [27]:
print(pa_ds24[pa_ds24["speciesId"] == 265].iloc[0])

lon                        6.92167
lat                      43.459111
year                          2018
geoUncertaintyInM              0.0
areaInM2                      50.0
region               MEDITERRANEAN
country                     France
speciesId                    265.0
surveyId                     17269
Name: 6038, dtype: object


In [ ]:
# Code-Snippet to match GLC23 occurrence with gbif occurrence to retrieve the original species
print(dataset_file[dataset_file["speciesId"] == 265].iloc[0])
#import pygbif
#pygbif.occurrences.get(key=3951621754)

In [29]:
pip install pygbif

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [30]:
import pygbif
pygbif.occurrences.get(key=3951621754)

{'key': 3951621754,
 'datasetKey': '14d5676a-2c54-4f94-9023-1e8dcd822aa0',
 'publishingOrgKey': 'da86174a-a605-43a4-a5e8-53d484152cd3',
 'installationKey': 'fe1a98a6-82b8-4652-86d6-d0f938207f67',
 'hostingOrganizationKey': 'da86174a-a605-43a4-a5e8-53d484152cd3',
 'publishingCountry': 'FR',
 'protocol': 'EML',
 'lastCrawled': '2023-02-08T16:42:42.739+00:00',
 'lastParsed': '2024-01-25T05:30:04.373+00:00',
 'crawlId': 129,
 'projectId': 'queries',
 'extensions': {},
 'basisOfRecord': 'HUMAN_OBSERVATION',
 'individualCount': 1,
 'occurrenceStatus': 'PRESENT',
 'taxonKey': 2879098,
 'kingdomKey': 6,
 'phylumKey': 7707728,
 'classKey': 220,
 'orderKey': 1354,
 'familyKey': 4689,
 'genusKey': 2877951,
 'speciesKey': 2879098,
 'acceptedTaxonKey': 2879098,
 'scientificName': 'Quercus ilex L.',
 'acceptedScientificName': 'Quercus ilex L.',
 'kingdom': 'Plantae',
 'phylum': 'Tracheophyta',
 'order': 'Fagales',
 'family': 'Fagaceae',
 'genus': 'Quercus',
 'species': 'Quercus ilex',
 'genericName'